In [1]:
get_ipython().system_raw("mlflow ui --port 5090 &")

In [2]:
!pip install pyngrok

Defaulting to user installation because normal site-packages is not writeable


[2025-02-06 08:08:38 +0000] [484] [INFO] Starting gunicorn 23.0.0
[2025-02-06 08:08:38 +0000] [484] [INFO] Listening at: http://127.0.0.1:5090 (484)
[2025-02-06 08:08:38 +0000] [484] [INFO] Using worker: sync
[2025-02-06 08:08:38 +0000] [485] [INFO] Booting worker with pid: 485
[2025-02-06 08:08:38 +0000] [486] [INFO] Booting worker with pid: 486
[2025-02-06 08:08:38 +0000] [487] [INFO] Booting worker with pid: 487
[2025-02-06 08:08:38 +0000] [488] [INFO] Booting worker with pid: 488


In [4]:
from pyngrok import ngrok

In [5]:
ngrok.kill()

In [6]:
!ngrok config add-authtoken 2sVwQYmf9x00hLx9OsT04T4Vk1R_3gyuN842eLrdSZYgebt5F

Authtoken saved to configuration file: /home/workbench/.config/ngrok/ngrok.yml                      


In [7]:
ngrok_tunnel = ngrok.connect(addr="5090", proto="http", bind_tls=True)

In [9]:
print("MLflow UI ", ngrok_tunnel.public_url)

MLflow UI  https://a769-67-254-220-141.ngrok-free.app


In [10]:
import torch
import flash
from flash.core.data.utils import download_data
from flash.text import TextClassificationData, TextClassifier

/home/workbench/.local/lib/python3.10/site-packages/flash/__init__.py:21: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/workbench/.local/lib/python3.10/site-packages/flash/__init__.py:21: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/workbench/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", "./data/")

In [12]:

datamodule = TextClassificationData.from_csv(

    input_field="review",

    target_fields="sentiment",

    train_file="data/imdb/train.csv",

    val_file="data/imdb/valid.csv",

    test_file="data/imdb/test.csv",
    batch_size=16

)

Map: 100%|██████████| 2500/2500 [00:00<00:00, 13203.79 examples/s]
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3550: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
pip install "numpy<2"

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 27.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.2
    Uninstalling numpy-2.2.2:
      Successfully uninstalled numpy-2.2.2
Note: you may need to restart the kernel to use updated packages.


In [15]:
classifier_model = TextClassifier(backbone="prajjwal1/bert-tiny", num_classes=datamodule.num_classes)
trainer = flash.Trainer(max_epochs=3, gpus=1)

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
import mlflow
EXPERIMENT_NAME = "dl_model_chapter12"
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)
mlflow.pytorch.autolog()
with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="chapter02"):
    trainer.finetune(classifier_model, datamodule=datamodule, strategy="freeze")
    trainer.test(datamodule=datamodule)

2025/02/06 08:16:09 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.5.1, but the installed version is 2.6.0+cu124. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.
INFO: You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated

experiment_id: 566112684550590904
Epoch 0:  90%|████████▉ | 1406/1563 [00:52<00:05, 26.79it/s, loss=0.634, v_num=3, train_accuracy_step=0.562, train_cross_entropy_step=0.654]
Validation: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 1563/1563 [00:57<00:00, 27.05it/s, loss=0.634, v_num=3, train_accuracy_step=0.562, train_cross_entropy_step=0.654, val_accuracy=0.634, val_cross_entropy=0.640, train_accuracy_epoch=0.601, train_cross_entropy_epoch=0.663]

2025/02/06 08:17:07 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


Epoch 1:  90%|████████▉ | 1406/1563 [00:50<00:05, 27.88it/s, loss=0.64, v_num=3, train_accuracy_step=0.750, train_cross_entropy_step=0.564, val_accuracy=0.634, val_cross_entropy=0.640, train_accuracy_epoch=0.601, train_cross_entropy_epoch=0.663] 
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 1563/1563 [00:55<00:00, 27.98it/s, loss=0.64, v_num=3, train_accuracy_step=0.750, train_cross_entropy_step=0.564, val_accuracy=0.651, val_cross_entropy=0.633, train_accuracy_epoch=0.623, train_cross_entropy_epoch=0.650]

2025/02/06 08:18:03 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


Epoch 2:  90%|████████▉ | 1406/1563 [00:52<00:05, 26.57it/s, loss=0.669, v_num=3, train_accuracy_step=0.500, train_cross_entropy_step=0.706, val_accuracy=0.651, val_cross_entropy=0.633, train_accuracy_epoch=0.623, train_cross_entropy_epoch=0.650]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████| 1563/1563 [00:58<00:00, 26.75it/s, loss=0.669, v_num=3, train_accuracy_step=0.500, train_cross_entropy_step=0.706, val_accuracy=0.660, val_cross_entropy=0.630, train_accuracy_epoch=0.625, train_cross_entropy_epoch=0.649]

2025/02/06 08:19:02 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.
INFO: `Trainer.fit` stopped: `max_epochs=3` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1563/1563 [00:58<00:00, 26.71it/s, loss=0.669, v_num=3, train_accuracy_step=0.500, train_cross_entropy_step=0.706, val_accuracy=0.660, val_cross_entropy=0.630, train_accuracy_epoch=0.625, train_cross_entropy_epoch=0.649]


2025/02/06 08:19:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
INFO: You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO: Restoring states from the checkpoin

Testing DataLoader 0: 100%|██████████| 157/157 [00:04<00:00, 38.72it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.6488000154495239
   test_cross_entropy       0.6284019947052002
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
